# gebrauchte Bücher aus Goodreads to-read liste suchen

In [2]:
import numpy as np
import csv

In [13]:
from matplotlib import pyplot as plt

## goodreads CSV-Datei parsen

In [56]:
import os
csv_path = os.path.normpath(r"C:\Users\simon\Downloads\goodreads_library_export.csv")
import pandas as pd

df = pd.read_csv(csv_path, index_col='Book Id',
dtype={"Title": str, "Author": str, "Author l-f": str, "Additional Authors": str, "ISBN":str,"ISBN-13":str, "Publisher": str, "Binding": str, 'Bookshelves': 'category', 'Exclusive Shelf': 'category', },
parse_dates=["Original Publication Year", 'Date Read', 'Date Added', "Year Published"])
df.head(5)

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,Number of Pages,Year Published,Original Publication Year,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
42242793,Kurze Antworten auf große Fragen,Stephen Hawking,"Hawking, Stephen",Hainer Kober,"=""""","=""""",0,4.30,Klett-Cotta,Kindle Edition,257.0,2018-01-01,2018,NaT,2021-02-07,currently-reading,currently-reading (#4),currently-reading,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
41160292,Exhalation: Stories,Ted Chiang,"Chiang, Ted",NaN,"=""""","=""""",4,4.27,Knopf,Kindle Edition,352.0,2019-01-01,2019,2021-02-07,2019-12-18,ebook,ebook (#30),read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
9778945,The Shallows: What the Internet Is Doing to Ou...,Nicholas Carr,"Carr, Nicholas",NaN,"=""0393339750""","=""9780393339758""",0,3.89,W. W. Norton Company,Paperback,280.0,2011-01-01,2010,NaT,2021-02-05,to-read,to-read (#245),to-read,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
23995249,The Invention of Nature: Alexander von Humbold...,Andrea Wulf,"Wulf, Andrea",NaN,"=""038535066X""","=""9780385350662""",0,4.24,Knopf,Hardcover,473.0,2015-01-01,2015,NaT,2021-02-05,to-read,to-read (#244),to-read,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
52668915,"Entangled Life: How Fungi Make Our Worlds, Cha...",Merlin Sheldrake,"Sheldrake, Merlin",NaN,"=""0525510311""","=""9780525510314""",0,4.42,Random House,Hardcover,352.0,2020-01-01,2020,NaT,2021-02-05,to-read,to-read (#243),to-read,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [57]:
df['ISBN'] = df['ISBN'].str.split('\"').str.get(1)
df['ISBN13'] = df['ISBN13'].str.split('\"').str.get(1)
df.head()

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,Number of Pages,Year Published,Original Publication Year,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
42242793,Kurze Antworten auf große Fragen,Stephen Hawking,"Hawking, Stephen",Hainer Kober,,,0,4.30,Klett-Cotta,Kindle Edition,257.0,2018-01-01,2018,NaT,2021-02-07,currently-reading,currently-reading (#4),currently-reading,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
41160292,Exhalation: Stories,Ted Chiang,"Chiang, Ted",NaN,,,4,4.27,Knopf,Kindle Edition,352.0,2019-01-01,2019,2021-02-07,2019-12-18,ebook,ebook (#30),read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
9778945,The Shallows: What the Internet Is Doing to Ou...,Nicholas Carr,"Carr, Nicholas",NaN,0393339750,9780393339758,0,3.89,W. W. Norton Company,Paperback,280.0,2011-01-01,2010,NaT,2021-02-05,to-read,to-read (#245),to-read,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
23995249,The Invention of Nature: Alexander von Humbold...,Andrea Wulf,"Wulf, Andrea",NaN,038535066X,9780385350662,0,4.24,Knopf,Hardcover,473.0,2015-01-01,2015,NaT,2021-02-05,to-read,to-read (#244),to-read,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
52668915,"Entangled Life: How Fungi Make Our Worlds, Cha...",Merlin Sheldrake,"Sheldrake, Merlin",NaN,0525510311,9780525510314,0,4.42,Random House,Hardcover,352.0,2020-01-01,2020,NaT,2021-02-05,to-read,to-read (#243),to-read,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [60]:
to_read = df[df['Exclusive Shelf'] == 'to-read']


In [64]:
import datetime as dt

In [69]:
to_read.sort_values('Date Added', ascending=False, inplace=True)
to_read_2021 = to_read[to_read['Date Added'] > pd.Timestamp('2020-01-01')]

In [70]:
to_read_2021.head()

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,Number of Pages,Year Published,Original Publication Year,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9778945,The Shallows: What the Internet Is Doing to Ou...,Nicholas Carr,"Carr, Nicholas",NaN,0393339750,9780393339758,0,3.89,W. W. Norton Company,Paperback,280.0,2011-01-01,2010,NaT,2021-02-05,to-read,to-read (#245),to-read,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
52668915,"Entangled Life: How Fungi Make Our Worlds, Cha...",Merlin Sheldrake,"Sheldrake, Merlin",NaN,0525510311,9780525510314,0,4.42,Random House,Hardcover,352.0,2020-01-01,2020,NaT,2021-02-05,to-read,to-read (#243),to-read,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
26150770,"The Big Picture: On the Origins of Life, Meani...",Sean Carroll,"Carroll, Sean",NaN,0525954821,9780525954828,0,4.18,Dutton,Hardcover,480.0,2016-01-01,2016,NaT,2021-02-05,to-read,to-read (#242),to-read,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
23995249,The Invention of Nature: Alexander von Humbold...,Andrea Wulf,"Wulf, Andrea",NaN,038535066X,9780385350662,0,4.24,Knopf,Hardcover,473.0,2015-01-01,2015,NaT,2021-02-05,to-read,to-read (#244),to-read,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
53687124,"Chasing the Light: Writing, Directing, and Sur...",Oliver Stone,"Stone, Oliver",NaN,,,0,4.07,Houghton Mifflin Harcourt,Kindle Edition,373.0,2020-01-01,2020,NaT,2021-02-01,to-read,to-read (#241),to-read,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


## Jetzt Suchmaschine mittels BeatifulSoup scrapen um Bestpreise für Bücher zu finden

In [86]:
from bs4 import BeautifulSoup
import requests

In [139]:
#Testbuch
book = to_read_2021.iloc[0]
book

Title                         The Shallows: What the Internet Is Doing to Ou...
Author                                                            Nicholas Carr
Author l-f                                                       Carr, Nicholas
Additional Authors                                                          NaN
ISBN                                                                 0393339750
ISBN13                                                            9780393339758
My Rating                                                                     0
Average Rating                                                             3.89
Publisher                                                 W. W. Norton  Company
Binding                                                               Paperback
Number of Pages                                                           280.0
Year Published                                              2011-01-01 00:00:00
Original Publication Year               

In [119]:
url = 'https://www.booklooker.de/B%C3%BCcher/Angebote/isbn=0393339750?sortOrder=preis_total'
search_url = 'https://www.booklooker.de/B%C3%BCcher/Angebote/'
base_url = 'https://www.booklooker.de'
query_parameters = {
    'isbn': book['ISBN13'],
    'sortOrder': 'preis_total'
}

In [94]:
page = requests.get(url)
soup = BeautifulSoup(page.content)

table class="resultlist_products"

tbody

tr id="article_*ID*" class="articleRow resultlist_products*product*" ODER

articleRow resultlist_products*premium*

In [97]:
results = soup.find(class_='resultlist_products')
products = results.find_all('tr', class_='articleRow')


In [202]:
entries = []
for idx, product in enumerate(products):
    product_info = product.find("td", class_="resultlist_productsimage")
    price_info = product.find("td", class_="resultlist_productsright")
    title_info =product_info.find('h2', class_='unformated')
    book_title = title_info.find('span', class_='artikeltitel')
    link = base_url + title_info.a.attrs['href']

    author = product_info.find('span', class_='notranslate')
    condition = str(product_info.find(string=lambda text: 'zustand' in text.lower()))
    prefix = 'Zustand: '
    condition = condition[len(prefix):] if condition.startswith(prefix) else condition
    price = price_info.find('span', class_='price')
    row = [book_title.text.strip(), author.text.strip(), price.text.strip()[:-2], condition.strip(), link]
    entries.append(row)
    # print(f"[{idx}]: title: {book_title.text.strip()}, author: {author.text.strip()}, price = {price.text.strip()}, state: {state.strip()}), link: {link}")


In [203]:
web = pd.DataFrame(entries, columns=['Title', 'Author', 'Price', 'Condition', 'URL'])
web.head()

,Title,Author,Price,Condition,URL
0,"Carr, N: Shallows","Carr, N: Shallows","3,21",gebraucht; mittelmäßig,https://www.booklooker.de/B%C3%BCcher/Nicholas...


Build Search URL

In [204]:
from urllib.parse import urlencode, urljoin, quote_plus
def build_booklooker_url(isbn):
    search_url = 'https://www.booklooker.de/B%C3%BCcher/Angebote/'
    payload = {'isbn': isbn, 'sortOrder': "preis_total"}
    result = urlencode(payload, quote_via=quote_plus)
    url = urllib.parse.urljoin(search_url, result)
    return url

def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content.decode('utf-8', 'ignore'))
    return soup

In [235]:
def get_booklooker_results(soup, limit=10, as_df=False):
    entries = []
    results = soup.find(class_='resultlist_products')
    if results is None:
        print('no resultlist found! returning.')
        return entries
    products = results.find_all('tr', class_='articleRow')
    for idx, product in enumerate(products):
        if idx > limit:
            break
        product_info = product.find("td", class_="resultlist_productsimage")
        title_info =product_info.find('h2', class_='unformated')
        book_title = title_info.find('span', class_='artikeltitel notranslate')
        link = base_url + title_info.a.attrs['href']

        author_info =product_info.find('h3', class_='unformated')
        author = author_info.find('span', class_='notranslate')

        condition = str(product_info.find(string=lambda text: 'zustand' in text.lower()))
        prefix = 'Zustand: '
        condition = condition[len(prefix):] if condition.startswith(prefix) else condition

        price_info = product.find("div", class_="productPrices")
        prices = price_info.find_all(string=lambda text: '€' in text.lower())
        price_list = [s[:-1].strip() for s in prices]
        price = price_list[0] if len(price_list) > 0 else ""
        vsk = price_list[1] if len(price_list) > 1 else ""
        
        row = [book_title.text.strip(), author.text.strip(), price, vsk, condition.strip(), link]
        entries.append(row)
    if as_df:
        return pd.DataFrame(entries, columns=['Title', 'Author', 'Price', 'VSK', 'Condition', 'URL'])
    else:
        return entries


## Bringing it all together

In [275]:
def get_sales_info_df(df, limit=10, as_df=False):
    isbn_ = df['ISBN13'] if len(df['ISBN13']) > 0 else df['ISBN']
    if len(isbn_) <= 0:
        print(f"{df['Title']}: no ISBN found!")
        return []
        # raise KeyError('No ISBN Found')
    url = build_booklooker_url(isbn_)
    soup = get_soup(url)
    search_results = get_booklooker_results(soup, limit=limit, as_df=as_df)
    return np.array(search_results)

In [236]:
def get_sales_info(title, isbn, isbn13, limit=10, as_df=False):
    isbn_ = isbn13 if len(isbn13) > 0 else isbn
    if len(isbn_) <= 0:
        print(f'{title}: no ISBN found!')
        return []
        # raise KeyError('No ISBN Found')
    url = build_booklooker_url(isbn_)
    soup = get_soup(url)
    search_results = get_booklooker_results(soup, limit=limit, as_df=as_df)
    return np.array(search_results)

In [276]:
from tqdm import tqdm
# sales_info = []
my_df = to_read_2021.iloc[:20]
from tqdm import tqdm
# Create and register a new `tqdm` instance with `pandas`
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

# Now you can use `progress_apply` instead of `apply`
sales_infos = my_df.progress_apply(get_sales_info_df, axis=1)


 35%|███▌      | 7/20 [00:01<00:03,  3.99it/s]no resultlist found! returning.
Die Daten, die ich rief: Wie wir unsere Freiheit an Großkonzerne verkaufen: no ISBN found!
100%|██████████| 20/20 [00:10<00:00,  1.97it/s]Zeit: Was sie mit uns macht und was wir aus ihr machen: no ISBN found!



In [285]:
sales_infos.head()
# nd = np.array(sales_infos)

Book Id
9778945     [[Carr, N: Shallows, Nicholas Carr, 3,21, 2,50...
52668915                                                   []
26150770    [[The Big Picture, Sean Carroll, 17,99, , None...
23995249    [[The Invention of Nature - Alexander von Humb...
53687124                                                   []
dtype: object

## TODO daraus jetzt ein MultiIndex basteln